In [1]:
# External imports
import torch
from torch.nn import functional as F
import torch.nn as nn
import lightning as L
from torchmetrics import classification, MetricCollection
import pandas as pd
import numpy as np
import os

# Internal imports
from utils.configurations import configs
from utils.logging_deepscreen import logger

class DEEPScreenClassifier(L.LightningModule):
    def __init__(self,fully_layer_1, fully_layer_2, drop_rate, learning_rate, batch_size, experiment_result_path, target):
        super(DEEPScreenClassifier, self).__init__()
        self.save_hyperparameters()
        logger.info(f"Using hyperparameters {[i for i in self.hparams.items()]}") 

        # Model architecture
        self.conv1 = nn.Conv2d(3, 32, 2)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 2)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 64, 2)
        self.bn4 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(64, 32, 2)
        self.bn5 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(1010, fully_layer_1)
        self.fc2 = nn.Linear(fully_layer_1, fully_layer_2)
        self.fc3 = nn.Linear(fully_layer_2, 2)
        self.drop_rate = drop_rate

        # Object atributes
        self.config = configs
        
        # Performance trackers
        self.train_metrics = MetricCollection(
             {
                "train_acc": classification.BinaryAccuracy(threshold=0.5),
                "train_prec": classification.BinaryPrecision(threshold=0.5),
                "train_f1": classification.BinaryF1Score(threshold=0.5),
                "train_mcc": classification.BinaryMatthewsCorrCoef(threshold=0.5),
                "train_recall": classification.BinaryRecall(threshold=0.5),
                "train_auroc": classification.BinaryAUROC(),
                "train_auroc_15": classification.BinaryAUROC(max_fpr=0.15),
                "train_calibration_error": classification.BinaryCalibrationError()
             }
         )
        
        self.val_metrics = MetricCollection(
             {
                "val_acc": classification.BinaryAccuracy(threshold=0.5),
                "val_prec": classification.BinaryPrecision(threshold=0.5),
                "val_f1": classification.BinaryF1Score(threshold=0.5),
                "val_mcc": classification.BinaryMatthewsCorrCoef(threshold=0.5),
                "val_recall": classification.BinaryRecall(threshold=0.5),
                "val_auroc": classification.BinaryAUROC(),
                "val_auroc_15": classification.BinaryAUROC(max_fpr=0.15),
                "val_calibration_error": classification.BinaryCalibrationError()
             }
         )
        
        self.test_metrics = MetricCollection(
             {
                "test_acc": classification.BinaryAccuracy(threshold=0.5),
                "test_prec": classification.BinaryPrecision(threshold=0.5),
                "test_f1": classification.BinaryF1Score(threshold=0.5),
                "test_mcc": classification.BinaryMatthewsCorrCoef(threshold=0.5),
                "test_recall": classification.BinaryRecall(threshold=0.5),
                "test_auroc": classification.BinaryAUROC(),
                "test_auroc_15": classification.BinaryAUROC(max_fpr=0.15),
                "test_calibration_error": classification.BinaryCalibrationError()
             }
         )

        # Predictions
        self.test_predictions = pd.DataFrame()
        self.predictions = pd.DataFrame()

    def forward(self, x, descriptors_features):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))
        x = x.view(-1, 32*5*5)
        mean = x.mean(dim=1, keepdim=True)
        std = x.std(dim=1, keepdim=True)
        descriptors_features_normalized = (descriptors_features - descriptors_features.mean(dim=0, keepdim=True)) / (descriptors_features.std(dim=0, keepdim=True) + 1e-6)
        descriptors_features_normalized = descriptors_features_normalized * std + mean
        x = torch.cat((x, descriptors_features_normalized), dim=1) 
        x = F.dropout(F.relu(self.fc1(x)), self.drop_rate, training = self.training)
        x = F.dropout(F.relu(self.fc2(x)), self.drop_rate, training = self.training)
        x = self.fc3(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),lr=self.hparams.learning_rate)
        return optimizer
    
    def cross_entropy_loss(self,input,labels):
        return F.cross_entropy(input,labels)
    
    def training_step(self,train_batch,batch_idx):
        img_arrs, label, comp_id, descriptor_features = train_batch
        y_pred = self.forward(img_arrs,descriptor_features)
        y_pred_soft_max = F.softmax(y_pred,dim=1)
        y_pred_soft_max_1_active = y_pred_soft_max[:,1]
        loss = self.cross_entropy_loss(y_pred.squeeze(),label)
        self.log('train_loss', loss, batch_size=self.hparams.batch_size, prog_bar=True, sync_dist=True)

        output = self.train_metrics(y_pred_soft_max_1_active,label.int())
        self.log_dict(output,on_step=False,on_epoch=True, batch_size=self.hparams.batch_size, sync_dist=True)
        
        return loss
      
    def validation_step(self,val_batch,batch_idx):
        img_arrs, label, comp_id, descriptor_features  = val_batch
        y_pred = self.forward(img_arrs, descriptor_features)
        y_pred_soft_max = F.softmax(y_pred,dim=1)
        y_pred_soft_max_1_active = y_pred_soft_max[:,1]
        loss = self.cross_entropy_loss(y_pred.squeeze(),label)
        self.log('val_loss', loss, batch_size=self.hparams.batch_size, prog_bar=True, sync_dist=True)

        output = self.val_metrics(y_pred_soft_max_1_active,label.int())
        self.log_dict(output,on_step=False,on_epoch=True,batch_size=self.hparams.batch_size,  sync_dist=True)

    def test_step(self,test_batch,batch_idx):
        img_arrs, label, comp_id, descriptor_features = test_batch
        y_pred = self.forward(img_arrs, descriptor_features)
        _, preds = torch.max(y_pred,1)
        y_pred_soft_max = F.softmax(y_pred,dim=1)
        y_pred_soft_max_1_active = y_pred_soft_max[:,1]
        loss = self.cross_entropy_loss(y_pred.squeeze(),label)
        self.log('test_loss', loss, batch_size=self.hparams.batch_size, prog_bar=True)

        output = self.test_metrics(y_pred_soft_max_1_active,label.int())
        self.log_dict(output,on_step=False,on_epoch=True,batch_size=self.hparams.batch_size)

        comp_id_pd = pd.Series(comp_id,name="comp_id")
        label_pd = pd.Series(label.cpu(),name="label")
        pred_pd = pd.Series(preds.cpu(),name="prediction")
        pred_0_pd = pd.Series(y_pred_soft_max[:,0].cpu(),name="0_inactive_probability")
        pred_1_pd = pd.Series(y_pred_soft_max[:,1].cpu(),name="1_active_probability")
        batch_predictions = pd.concat([comp_id_pd,label_pd,pred_pd,pred_0_pd,pred_1_pd],axis=1)
        self.test_predictions = pd.concat([self.test_predictions,batch_predictions],axis=0)

    def on_test_end(self):
        self.test_predictions.to_csv(os.path.join(self.hparams.experiment_result_path,f"test_{self.hparams.target}_{self.hparams.fully_layer_1}-{self.hparams.fully_layer_2}-{self.hparams.learning_rate}-{self.hparams.drop_rate}-{self.hparams.batch_size}.csv"),index=False)

    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        img_arrs, comp_id, descriptor_features = batch
        y_pred = self.forward(img_arrs, descriptor_features)
        _, preds = torch.max(y_pred,1)
        comp_id_pd = pd.Series(comp_id,name="comp_id")
        pred_pd = pd.Series(preds.cpu(),name="prediction")
        y_pred_soft_max = F.softmax(y_pred,dim=1) 
        pred_0_pd = pd.Series(y_pred_soft_max[:,0].cpu(),name="0_inactive_probability")
        pred_1_pd = pd.Series(y_pred_soft_max[:,1].cpu(),name="1_active_probability")
        batch_predictions = pd.concat([comp_id_pd,pred_pd,pred_0_pd,pred_1_pd],axis=1)
        self.predictions = pd.concat([self.predictions,batch_predictions],axis=0)
        return batch_predictions

    def on_predict_epoch_end(self):
        return self.predictions
    
    def on_predict_end(self):
        self.predictions.to_csv(os.path.join(self.hparams.experiment_result_path,f"predictions_{self.hparams.target}_{self.hparams.fully_layer_1}-{self.hparams.fully_layer_2}-{self.hparams.learning_rate}-{self.hparams.drop_rate}-{self.hparams.batch_size}.csv"),index=False)

/home/sjinich/disco/che_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-27 16:42:28,272	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-27 16:42:28,372	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# External imports
import os
import pandas as pd
import numpy as np
import cv2
import random
from torch.utils.data import Dataset
import torch
from rdkit.Chem import Draw, MolFromSmiles
import re
import deepchem as dc

# Internal Imports
from utils.constants import RANDOM_STATE
from utils.logging_deepscreen import logger
from utils.configurations import configs

random.seed(RANDOM_STATE)

class DEEPScreenDataset(Dataset):
    def __init__(self, path_imgs_files:str, df_compid_smiles_bioactivity:pd.DataFrame):
            super(DEEPScreenDataset, self).__init__()

            self.path_imgs = path_imgs_files
            self.df = df_compid_smiles_bioactivity.copy()
            self.config = configs

            if not os.path.exists(self.path_imgs):
                os.makedirs(self.path_imgs)

            # creating molecules images -> path will be stored in 'img_molecule' column
            self.df['img_molecule'] = self.df.apply(lambda x: self.smiles_to_img_png(x["comp_id"],x["smiles"],self.path_imgs),axis=1)

            featurizer = dc.feat.RDKitDescriptors()
            features = featurizer.featurize(self.df["smiles"])
            descriptors_rdkit = pd.Series([torch.tensor(tensor).type(torch.FloatTensor) for tensor in features], name="descriptor_features")
            self.df = pd.concat([self.df,descriptors_rdkit],axis=1)

            logger.debug(f'Dataset created in {self.path_imgs}')


    def __len__(self):
        return len(self.df.index)
    
    def smiles_to_img_png(self, comp_id:str, smiles:str, output_path:str)->str:
        '''
        Given an id and an output path the function will create a image with the 2D molecule. 

        Returns: the path to the file i.e. /output_path/comp_id.png
        '''
        mol = MolFromSmiles(smiles)
        opt = self.config.get_mol_draw_options()
        img_size = self.config.get_img_size()
        comp_id_clean = re.sub('[^A-Za-z0-9]+', '_', comp_id)

        output_file = os.path.join(output_path, f"{comp_id_clean}.png")
        Draw.MolToFile(mol, output_file, size=img_size, options=opt)
        return output_file


class DEEPScreenDatasetPredict(DEEPScreenDataset):
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        comp_id = row["comp_id"]
        img_path = row['img_molecule']
        features = row["descriptor_features"]
        img_arr = cv2.imread(img_path)
        img_arr = np.array(img_arr) / 255.0
        img_arr = img_arr.transpose((2, 0, 1))
        return torch.tensor(img_arr).type(torch.FloatTensor), comp_id, features
    

class DEEPScreenDatasetTrain(DEEPScreenDataset):

    def __getitem__(self, index):
        row = self.df.iloc[index]
        comp_id = row["comp_id"]
        img_path = row['img_molecule']
        features = row["descriptor_features"]
        img_arr = cv2.imread(img_path)
        if random.random()>=0.50:
            angle = random.randint(0,359)
            rows, cols, channel = img_arr.shape
            rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
            img_arr = cv2.warpAffine(img_arr, rotation_matrix, (cols, rows), cv2.INTER_LINEAR,
                                             borderValue=(255, 255, 255))  # cv2.BORDER_CONSTANT, 255)
        img_arr = np.array(img_arr) / 255.0
        img_arr = img_arr.transpose((2, 0, 1))
        label = row["bioactivity"]

        return torch.tensor(img_arr).type(torch.FloatTensor), torch.tensor(label).type(torch.LongTensor), comp_id, features
    

class DEEPScreenDatasetTest(DEEPScreenDataset):

    def __getitem__(self, index):
        row = self.df.iloc[index]
        comp_id = row["comp_id"]
        img_path = row['img_molecule']
        features = row["descriptor_features"]
        img_arr = cv2.imread(img_path)
        img_arr = np.array(img_arr) / 255.0
        img_arr = img_arr.transpose((2, 0, 1))
        label = row["bioactivity"]

        return torch.tensor(img_arr).type(torch.FloatTensor), torch.tensor(label).type(torch.LongTensor), comp_id, features

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/sjinich/disco/che_env/lib/python3.8/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
# External imports
import lightning as L
import pandas as pd
import os
from torch.utils.data import DataLoader
import tempfile

# Internal imports
from utils.logging_deepscreen import logger
from utils.exceptions import InvalidDataframeException
from utils.configurations import configs


class DEEPscreenDataModule(L.LightningDataModule):
    
    def __init__(self, data:str, batch_size:int, experiment_result_path:str, data_split_mode:str, tmp_imgs:bool = False):

        super(DEEPscreenDataModule, self).__init__()

        self.save_hyperparameters()
    
        self.result_path = experiment_result_path

        self.batch_size = batch_size

        if tmp_imgs:
            self.imgs_path = tempfile.TemporaryDirectory().name
        else:
            self.imgs_path = os.path.join(self.result_path,"imgs")
        
        if data_split_mode in ("random_split","non_random_split","scaffold_split","predict"):
            self.data_split = data_split_mode
        else:
            raise Exception("data split mode should be one of random_split/non_random_split/scaffold_split/predict")

        self.data = data

        if data_split_mode != "predict" and not {"comp_id","smiles","bioactivity"}.issubset(set(self.data.columns)):
            logger.error("invalid columns of df")
            raise InvalidDataframeException("must contain the following columns {'comp_id','smiles','bioactivity'}")

    def setup(self,stage:str):
        
        # cleaning data to prune posible errors
        self.data = self.data.dropna(how="any")

        logger.info(f"Using a total of {len(self.data)} datapoints")

        if stage == "fit" or stage == "test" or stage == "validation":

            # sanitizeing data
            self.data["bioactivity"] = self.data["bioactivity"].astype(int)

            if self.data_split == "random_split":
                #TODO
                raise NotImplementedError

            if self.data_split == "non_random_split":
                dataset = self._non_random_split(self.data)
                self.train = dataset["train"]
                self.validate = dataset["validation"]
                self.test = dataset["test"]

            if self.data_split == "scaffold_split":
                #TODO
                raise NotImplementedError
            
        
        if stage == "predict":
            self.predict = self.data
    
    def get_number_training_batches(self):
        if self.data_split == "non_random_split":
            number_training_batches = round(len(self.data[self.data["data_split"] == "train"])/self.batch_size)
        else:
            number_training_batches = 50
            
        return number_training_batches
    
    def _non_random_split(self,data):
        if "data_split" not in data.columns:
            logger.error("theres not a 'data_split' column in the dataframe for non random datasplit")
            raise InvalidDataframeException("Missing 'data_split' column while using non_random_split")

        if not set(data["data_split"].unique()).issubset({"train","validation","test","predict"}):
            logger.error("invalid tags or missing tags for data spliting in non random datasplit")
            raise InvalidDataframeException("Invalid tags or missing tags for data spliting in non random datasplit, tags should be in ('trian','validation','test','predict')")
        
        dataset = {"train":None,"validation":None,"test":None}
        
        try:
            for key in data["data_split"].unique():
                dataset[key] = data[data["data_split"]==key]
                logger.info(f"non_random_split dataset splited {key}={len(dataset[key])}")
            
        except Exception as e:
            logger.error(f"Unable to create non_random_split datasets {e}")
            raise RuntimeError("non_random_split dataloader type generator failed")

        return dataset


    def train_dataloader(self):
        self.training_dataset = DEEPScreenDatasetTrain(self.imgs_path, self.train)
        return DataLoader(self.training_dataset,batch_size=self.hparams.batch_size,shuffle=True)
    
    def val_dataloader(self):
        self.validation_dataset = DEEPScreenDatasetTest(self.imgs_path, self.validate)
        return DataLoader(self.validation_dataset,batch_size=self.hparams.batch_size)
    
    def test_dataloader(self):
        self.test_dataset = DEEPScreenDatasetTest(self.imgs_path, self.test)
        return DataLoader(self.test_dataset,batch_size=self.hparams.batch_size)
    
    def predict_dataloader(self):
        self.predict_dataset = DEEPScreenDatasetPredict(self.imgs_path, self.predict)
        return DataLoader(self.predict_dataset,batch_size=self.hparams.batch_size)
        


In [4]:
from lightning import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint

In [5]:
%load_ext tensorboard


In [6]:
df = pd.read_csv("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL2581.csv")

In [8]:
trainer = Trainer(max_epochs=100)
model = DEEPScreenClassifier(fully_layer_1=256,fully_layer_2=32,drop_rate=0.5,learning_rate=0.0001,batch_size=32,experiment_result_path="../../.experiments/chembl2581",target="CHEMBL2581")
datamodule = DEEPscreenDataModule(data=df,batch_size=32,experiment_result_path="../../.experiments/chembl2581",data_split_mode="non_random_split",tmp_imgs=True)
trainer.fit(model,datamodule=datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
INFO: Using hyperparameters [('fully_layer_1', 256), ('fully_layer_2', 32), ('drop_rate', 0.5), ('learning_rate', 0.0001), ('batch_size', 32), ('experiment_result_path', '../../.experiments/chembl2581'), ('target', 'CHEMBL2581')]


INFO: Using a total of 2270 datapoints
INFO: non_random_split dataset splited validation=364
INFO: non_random_split dataset splited train=1452
INFO: non_random_split dataset splited test=454

   | Name          | Type             | Params
----------------------------------------------------
0  | conv1         | Conv2d           | 416   
1  | bn1           | BatchNorm2d      | 64    
2  | conv2         | Conv2d           | 8.3 K 
3  | bn2           | BatchNorm2d      | 128   
4  | conv3         | Conv2d           | 32.9 K
5  | bn3           | BatchNorm2d      | 256   
6  | conv4         | Conv2d           | 32.8 K
7  | bn4           | BatchNorm2d      | 128   
8  | conv5         | Conv2d           | 8.2 K 
9  | bn5           | BatchNorm2d      | 64    
10 | pool          | MaxPool2d        | 0     
11 | fc1           | Linear           | 258 K 
12 | fc2           | Linear           | 8.2 K 
13 | fc3           | Linear           | 66    
14 | train_metrics | MetricCollection | 0     
15 

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/sjinich/disco/che_env/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 800 but got size 210 for tensor number 1 in the list.

In [10]:
img_arr = cv2.imread("/home/sjinich/disco/TrypanoDEEPscreen/.experiments/imgs/2_Deoxy_2_methyl_nitroso_carbamoyl_amino_hexose.png")
if random.random()>=0:
    angle = random.randint(0,359)
    rows, cols, channel = img_arr.shape
    rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    img_arr = cv2.warpAffine(img_arr, rotation_matrix, (cols, rows), cv2.INTER_LINEAR,
                                     borderValue=(255, 255, 255))  # cv2.BORDER_CONSTANT, 255)
img_arr = np.array(img_arr) / 255.0
img_arr = img_arr.transpose((2, 0, 1))
tensor = torch.tensor([img_arr]).type(torch.FloatTensor)

network = DEEPScreenClassifier(512,256,0.001,0.3,32,"/home/sjinich/disco/TrypanoDEEPscreen/.experiments/trial_adding_vector","trial_adding_tensor")

otuput, lineal_tensor = network.forward(tensor, df_d.loc[0,"descriptor_features"])

/tmp/ipykernel_206608/2513407159.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  tensor = torch.tensor([img_arr]).type(torch.FloatTensor)
INFO: Using hyperparameters [('fully_layer_1', 512), ('fully_layer_2', 256), ('drop_rate', 0.001), ('learning_rate', 0.3), ('batch_size', 32), ('experiment_result_path', '/home/sjinich/disco/TrypanoDEEPscreen/.experiments/trial_adding_vector'), ('target', 'trial_adding_tensor')]


NameError: name 'df_d' is not defined

In [118]:
otuput.squeeze()

tensor([0.0088, 0.1532], grad_fn=<SqueezeBackward0>)

In [8]:
featurizer = dc.feat.RDKitDescriptors()
features = featurizer.featurize(df["smiles"])
descriptors_rdkit = pd.Series([torch.tensor(tensor).type(torch.FloatTensor) for tensor in features], name="descriptor_features")
df = pd.concat([df,descriptors_rdkit],axis=1)


In [14]:
df.iloc[0]["descriptor_features"]

tensor([ 1.4299e+01,  1.4299e+01,  2.0595e-02, -1.5042e+00,  7.5694e-02,
         1.3786e+01,  7.6595e+02,  7.1051e+02,  7.6541e+02,  2.9800e+02,
         0.0000e+00,  4.0763e-01, -4.9677e-01,  4.9677e-01,  4.0763e-01,
         5.7143e-01,  1.0714e+00,  1.6250e+00,  1.6548e+01,  9.9325e+00,
         2.3755e+00, -2.3335e+00,  2.1566e+00, -2.6294e+00,  5.9045e+00,
        -1.3356e-01,  3.2064e+00,  1.5823e+00,  1.8082e+03,  4.0451e+01,
         3.2736e+01,  3.2736e+01,  2.6943e+01,  1.8926e+01,  1.8926e+01,
         1.4363e+01,  1.4363e+01,  9.3392e+00,  9.3392e+00,  6.0694e+00,
         6.0694e+00, -5.7200e+00,  3.2064e+00,  4.3014e+01,  2.1987e+01,
         1.3540e+01,  3.2932e+02,  3.5847e+01,  3.0482e+01,  0.0000e+00,
         1.7722e+01,  0.0000e+00,  6.0932e+00,  1.9700e+01,  4.7945e+00,
         0.0000e+00,  0.0000e+00,  1.3082e+02,  5.2643e+01,  1.3090e+01,
         1.9256e+01,  3.3758e+01,  2.3815e+01,  0.0000e+00,  2.6584e+01,
         1.1836e+01,  8.4083e+01,  7.1098e+00,  1.3